In [ ]:
#importing packages
import cv2
import numpy as np
import os #help to interact with theoperating system
import matplotlib.pyplot as plt
import pickle #help to zip/compress the img file in byte(binary) format

In [ ]:
#accessing the dataset present inside "Emotion":
#getcwd :- fetching the current working directory
img_dir = os.path.join(os.getcwd(),"Emotion") #joins the path of the os and our current working path

In [ ]:
#to check the directory where currently we are working:-
print(os.getcwd())

In [ ]:
#creating a path to store the picl;e files inside data folder.
img_final = os.path.join(os.getcwd(),"data")

In [ ]:
#to store all input (images) and output(labels )in different []
images=[]
labels =[]

In [ ]:
#performing mage pre processing and saving the files.
# listdir: to list all folders/file present in any directory
for i in os.listdir(img_dir): #go inside the emotion folder in order to access each sub folder(sad,angry,happy)
    for j in os.listdir(os.path.join(img_dir,i)):
      img_path = os.path.join(os.path.join(img_dir,i),j)
      img=cv2.imread(img_path)
      img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
      img=cv2.resize(img,(48,48)) #since max images are 48x48
      images.append(img)
      labels.append(i.split(" ")[0])
    print(i, '---> Completed')

In [ ]:
set(labels)

In [ ]:
images =np.array(images)
labels = np.array(labels)

In [ ]:
images.shape

In [ ]:
#to plot an image after pre-preprocessing
plt.imshow(images[1000],cmap='gray')
plt.show()

In [ ]:
#Pickling the images and labels files
with open(os.path.join(img_final,'images.p'),'wb') as f:
    pickle.dump(images,f)
with open(os.path.join(img_final,'labels.p'),'wb') as f:
    pickle.dump(labels,f)


#MODEL TRAINING

In [ ]:
import pickle
import numpy as np
import tensorflow as tf
import keras


In [ ]:
#Loading the data from pickle file:
with open("images.p",'rb') as f:
    x = pickle.load(f)
with open("labels.p",'rb') as f:
    y = pickle.load(f)

In [ ]:
print("Images: ",x.shape)
print("Labels: ",y.shape)

Images:  (16082, 48, 48)
Labels:  (16082,)


In [ ]:
#Label encoding output
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
y

array([0, 0, 0, ..., 4, 4, 4])

In [ ]:
#Standardization the images for CNN model:
x=x/255
x=x.reshape(-1,48,48,1)

In [ ]:
#Using one-hot encoding to output col:
from keras.utils import to_categorical
y=to_categorical(y)

In [ ]:
#Splitting training and testing datas:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest= train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
print("Xtrain: ",xtrain.shape)
print("Xtest: ",xtest.shape)
print("Ytrain: ",ytrain.shape)
print("Ytest: ",ytest.shape)

Xtrain:  (12865, 48, 48, 1)
Xtest:  (3217, 48, 48, 1)
Ytrain:  (12865, 5)
Ytest:  (3217, 5)


In [ ]:
##CNN Model Training

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [ ]:
model=Sequential()

#1st convolutional layer:
model.add(Conv2D(45,(3,3),activation='relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))

#2nd convolutional layer:
model.add(Conv2D(65,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#3rd convolutional layer:
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Adding flatten layer : converts 2D to 1D
model.add(Flatten())

#Adding hidden layer
model.add(Dense(256,activation='relu'))

#Output layer
model.add(Dense(5,activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])



/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 45)     │           450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 45)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 65)     │        26,390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 65)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        75,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 627,677 (2.39 MB)

 Trainable params: 627,677 (2.39 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(xtrain,ytrain,epochs=30,validation_data=(xtest,ytest))

Epoch 1/30
403/403 ━━━━━━━━━━━━━━━━━━━━ 64s 154ms/step - accuracy: 0.3115 - loss: 1.5240 - val_accuracy: 0.4653 - val_loss: 1.2967
Epoch 2/30
403/403 ━━━━━━━━━━━━━━━━━━━━ 83s 156ms/step - accuracy: 0.4639 - loss: 1.3007 - val_accuracy: 0.5185 - val_loss: 1.1764
Epoch 3/30
403/403 ━━━━━━━━━━━━━━━━━━━━ 83s 158ms/step - accuracy: 0.5350 - loss: 1.1456 - val_accuracy: 0.5421 - val_loss: 1.1125
Epoch 4/30
403/403 ━━━━━━━━━━━━━━━━━━━━ 63s 157ms/step - accuracy: 0.5777 - loss: 1.0419 - val_accuracy: 0.5894 - val_loss: 1.0353
Epoch 5/30
403/403 ━━━━━━━━━━━━━━━━━━━━ 83s 161ms/step - accuracy: 0.6229 - loss: 0.9540 - val_accuracy: 0.5769 - val_loss: 1.0921
Epoch 6/30
403/403 ━━━━━━━━━━━━━━━━━━━━ 63s 156ms/step - accuracy: 0.6345 - loss: 0.9442 - val_accuracy: 0.6158 - val_loss: 1.0007
Epoch 7/30
403/403 ━━━━━━━━━━━━━━━━━━━━ 83s 158ms/step - accuracy: 0.6930 - loss: 0.7894 - val_accuracy: 0.6108 - val_loss: 1.0194
Epoch 8/30
403/403 ━━━━━━━━━━━━━━━━━━━━ 62s 154ms/step - accuracy: 0.7215 - loss: 0

In [ ]:
#to save the model for deployment
#h5 models represent HDF5 Format
# It stores model's optimizer's state,model architecture,training configuration,weights
model.save('emotion_detection_model.h5')
print("Model Saved")

Model Saved
